In [1]:
from sklearn import svm, metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
from imblearn.over_sampling import SMOTE

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
accidents = pd.read_csv('../data/MA3_finished.csv', index_col='ID')
accidents = accidents.dropna()
accidents = accidents.sample(n=70000, random_state=1)

In [ ]:
print(accidents.describe())

#count = Anzahl der Daten
#mean = Mittelwert
#std = Standardabweichung
#Min & Max werte
#Quartile (25%, 50%, 75%):

In [ ]:
#featureselection
X = accidents[['Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',
               'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit',
               'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]

y = accidents['Severity']

#Encoding text/categorical variables
le = LabelEncoder()
X.loc[:,'Weather_Condition']= le.fit_transform(X['Weather_Condition'])

print(X.describe())

In [ ]:
# Feature-Selection und SVC-Classifier
dim_reduction = VarianceThreshold(0.1)
classifier = svm.SVC()

# Trainings- und Testdaten aufteilen
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.5, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_test1, y_test1, test_size=0.5, random_state=42)

print("X_train Shape:", X_train.shape)
print("y_train Shape:", y_train.shape)

In [ ]:
#Balancing severity für recall, precision
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
print(accidents.columns)

In [ ]:
# Pipeline erstellen
pipeline = Pipeline([
    ('dim_reduction', dim_reduction),
    ('classifier', classifier)
])

# Trainieren der Pipeline
pipeline.fit(X_train, y_train)

# Vorhersagen machen
predicted_dev = pipeline.predict(X_dev)

# Varianz der einzelnen Features ausgeben
print("Varianzen der Features nach VarianceThreshold:")
print(dim_reduction.variances_)
print("-----------------------------------------------------------------------")
# Bericht für die Pipeline ausgeben
print("Classification Report für die Pipeline (VarianceThreshold + SVC):")
print(metrics.classification_report(y_dev, predicted_dev))
